In [2]:
import os
import soundfile as sf
import time
import math
import numpy as np
from natsort import natsorted
import soundfile as sf
import librosa
import h5py

In [3]:
files_FA = natsorted(np.squeeze([x[2] for x in os.walk('TSP/48k/FA')]))
files_FB = natsorted(np.squeeze([x[2] for x in os.walk('TSP/48k/FB')]))
files_MC = natsorted(np.squeeze([x[2] for x in os.walk('TSP/48k/MC')]))
files_MD = natsorted(np.squeeze([x[2] for x in os.walk('TSP/48k/MD')]))

# Functions

In [4]:
def STFT(wavefile):
    N = 2048
    M = 512
    frames = librosa.stft(wavefile, n_fft=2048, hop_length=M, win_length=N, window='hann', center=False)
    data_phase=np.angle(frames)
    return frames, data_phase

# For 1 hour of training data we need 800 mixed files of each type

80% of all data is for training - 10% valid - 10% test

FA VS MC

In [5]:
mixed_number = 0
chosen = []
counter = 0
file_series = str(0)
while(mixed_number < 800):
    while(True):
        file1_indx = np.random.randint(0,(len(files_FA)-1) * 0.8) # 80%
        file2_indx = np.random.randint(0,(len(files_MC)-1) * 0.8) # 80%
        if str(file1_indx) + str('_') + str(file2_indx) not in chosen: # to prevent from choosing repeated files
            chosen.append(str(file1_indx) + str('_') + str(file2_indx))
            mixed_number += 1
            break
    wave1, samplerate = sf.read('TSP/48k/FA/' + files_FA[file1_indx])
    wave2, samplerate = sf.read('TSP/48k/MC/' + files_MC[file2_indx])
    if len(wave1) < len(wave2): # cropping data
        mixed = wave1 + wave2[0:len(wave1)]
        wave2 = wave2[0:len(wave1)]
    else:
        mixed = wave2 + wave1[0:len(wave2)]
        wave1 = wave1[0:len(wave2)]
    # framming and FFT and phase
    mixed_FFT, mixed_phase = STFT(mixed)
    wave1_FFT, wave1_phase = STFT(wave1)
    wave2_FFT, wave2_phase = STFT(wave2)  
    # concate wave 1 and wave 2 to have a 2 times dimention of each
    clean_FFTs = np.concatenate((wave1_FFT, wave2_FFT), axis = 0)
    clean_phases = np.concatenate((wave1_phase, wave2_phase), axis = 0)
    
    # writing files of TSP_mixed_FFT **** TSP_mixed_phase **** TSP_clean_FFTs **** TSP_clean_phases
    if counter == 0:
        with h5py.File('TSP/Organized/concatenated/Train_TSP_mixed_FFT' + file_series + '.hdf5', "w") as f_mixed_FFT:
                dset = f_mixed_FFT.create_dataset('Train_TSP_mixed_FFT' + file_series ,shape=(mixed_FFT.shape[0],0), maxshape=(mixed_FFT.shape[0],100000000),
                                chunks=True)
        with h5py.File('TSP/Organized/concatenated/Train_TSP_mixed_phase' + file_series + '.hdf5', "w") as f_mixed_phase:
                dset = f_mixed_phase.create_dataset('Train_TSP_mixed_phase' + file_series ,shape=(mixed_phase.shape[0],0), maxshape=(mixed_phase.shape[0],100000000),
                                chunks=True)
        with h5py.File('TSP/Organized/concatenated/Train_TSP_clean_FFTs' + file_series + '.hdf5', "w") as f_clean_FFTs:
                dset = f_clean_FFTs.create_dataset('Train_TSP_clean_FFTs' + file_series ,shape=(clean_FFTs.shape[0],0), maxshape=(clean_FFTs.shape[0],100000000),
                                chunks=True)
        with h5py.File('TSP/Organized/concatenated/Train_TSP_clean_phases' + file_series + '.hdf5', "w") as f_clean_phases:
                dset = f_clean_phases.create_dataset('Train_TSP_clean_phases' + file_series ,shape=(clean_phases.shape[0],0), maxshape=(clean_phases.shape[0],100000000),
                                chunks=True)

    # mixed_FFT
    f_mixed_FFT = h5py.File('TSP/Organized/concatenated/Train_TSP_mixed_FFT' + file_series + '.hdf5', 'r+')     # open the file
    dset = f_mixed_FFT['Train_TSP_mixed_FFT' + file_series]                # load the data
    dset_shape = dset.shape
    dset.resize(dset_shape[1] + mixed_FFT.shape[1], axis=1)  
    dset[:,dset_shape[1]:] = np.abs(mixed_FFT)
    f_mixed_FFT.close() 
    
    # mixed_phase
    f_mixed_phase = h5py.File('TSP/Organized/concatenated/Train_TSP_mixed_phase' + file_series + '.hdf5', 'r+')     # open the file
    dset = f_mixed_phase['Train_TSP_mixed_phase' + file_series]                # load the data
    dset_shape = dset.shape
    dset.resize(dset_shape[1] + mixed_phase.shape[1], axis=1)  
    dset[:,dset_shape[1]:] = np.abs(mixed_phase)
    f_mixed_phase.close() 
    
    # clean_FFTs
    f_clean_FFTs = h5py.File('TSP/Organized/concatenated/Train_TSP_clean_FFTs' + file_series + '.hdf5', 'r+')     # open the file
    dset = f_clean_FFTs['Train_TSP_clean_FFTs' + file_series]                # load the data
    dset_shape = dset.shape
    dset.resize(dset_shape[1] + clean_FFTs.shape[1], axis=1)  
    dset[:,dset_shape[1]:] = np.abs(clean_FFTs)
    f_clean_FFTs.close() 
    
    # clean_phases
    f_clean_phases = h5py.File('TSP/Organized/concatenated/Train_TSP_clean_phases' + file_series + '.hdf5', 'r+')     # open the file
    dset = f_clean_phases['Train_TSP_clean_phases' + file_series]                # load the data
    dset_shape = dset.shape
    dset.resize(dset_shape[1] + clean_phases.shape[1], axis=1)  
    dset[:,dset_shape[1]:] = np.abs(clean_phases)
    f_clean_phases.close() 
    

    counter += 1


In [6]:
with open('TSP/Organized/concatenated/Train_selected_FA_MC.txt', 'w') as f:
    for item in chosen:
        f.write("%s\n" % item)

In [7]:
a = h5py.File('TSP/Organized/concatenated/Train_TSP_clean_FFTs'  + file_series + '.hdf5','r')
data_shape = a['Train_TSP_clean_FFTs' + file_series].shape
print(data_shape)
a.close()

(2050, 154781)


FA VS FB

In [8]:
mixed_number = 0
chosen = []
counter = 0
file_series = str(0)
while(mixed_number < 800):
    while(True):
        file1_indx = np.random.randint(0,(len(files_FA)-1) * 0.8) # 80%
        file2_indx = np.random.randint(0,(len(files_FB)-1) * 0.8) # 80%
        if str(file1_indx) + str('_') + str(file2_indx) not in chosen: # to prevent from choosing repeated files
            chosen.append(str(file1_indx) + str('_') + str(file2_indx))
            mixed_number += 1
            break
    wave1, samplerate = sf.read('TSP/48k/FA/' + files_FA[file1_indx])
    wave2, samplerate = sf.read('TSP/48k/FB/' + files_FB[file2_indx])
    if len(wave1) < len(wave2): # cropping data
        mixed = wave1 + wave2[0:len(wave1)]
        wave2 = wave2[0:len(wave1)]
    else:
        mixed = wave2 + wave1[0:len(wave2)]
        wave1 = wave1[0:len(wave2)]
    # framming and FFT and phase
    mixed_FFT, mixed_phase = STFT(mixed)
    wave1_FFT, wave1_phase = STFT(wave1)
    wave2_FFT, wave2_phase = STFT(wave2)  
    # concate wave 1 and wave 2 to have a 2 times dimention of each
    clean_FFTs = np.concatenate((wave1_FFT, wave2_FFT), axis = 0)
    clean_phases = np.concatenate((wave1_phase, wave2_phase), axis = 0)
    
    # writing files of TSP_mixed_FFT **** TSP_mixed_phase **** TSP_clean_FFTs **** TSP_clean_phases

    # mixed_FFT
    f_mixed_FFT = h5py.File('TSP/Organized/concatenated/Train_TSP_mixed_FFT' + file_series + '.hdf5', 'r+')     # open the file
    dset = f_mixed_FFT['Train_TSP_mixed_FFT' + file_series]                # load the data
    dset_shape = dset.shape
    dset.resize(dset_shape[1] + mixed_FFT.shape[1], axis=1)  
    dset[:,dset_shape[1]:] = np.abs(mixed_FFT)
    f_mixed_FFT.close() 
    
    # mixed_phase
    f_mixed_phase = h5py.File('TSP/Organized/concatenated/Train_TSP_mixed_phase' + file_series + '.hdf5', 'r+')     # open the file
    dset = f_mixed_phase['Train_TSP_mixed_phase' + file_series]                # load the data
    dset_shape = dset.shape
    dset.resize(dset_shape[1] + mixed_phase.shape[1], axis=1)  
    dset[:,dset_shape[1]:] = np.abs(mixed_phase)
    f_mixed_phase.close() 
    
    # clean_FFTs
    f_clean_FFTs = h5py.File('TSP/Organized/concatenated/Train_TSP_clean_FFTs' + file_series + '.hdf5', 'r+')     # open the file
    dset = f_clean_FFTs['Train_TSP_clean_FFTs' + file_series]                # load the data
    dset_shape = dset.shape
    dset.resize(dset_shape[1] + clean_FFTs.shape[1], axis=1)  
    dset[:,dset_shape[1]:] = np.abs(clean_FFTs)
    f_clean_FFTs.close() 
    
    # clean_phases
    f_clean_phases = h5py.File('TSP/Organized/concatenated/Train_TSP_clean_phases' + file_series + '.hdf5', 'r+')     # open the file
    dset = f_clean_phases['Train_TSP_clean_phases' + file_series]                # load the data
    dset_shape = dset.shape
    dset.resize(dset_shape[1] + clean_phases.shape[1], axis=1)  
    dset[:,dset_shape[1]:] = np.abs(clean_phases)
    f_clean_phases.close() 
    

    counter += 1


In [9]:
with open('TSP/Organized/concatenated/Train_selected_FA_FB.txt', 'w') as f:
    for item in chosen:
        f.write("%s\n" % item)

In [10]:
a = h5py.File('TSP/Organized/concatenated/Train_TSP_clean_FFTs'  + file_series + '.hdf5','r')
data_shape = a['Train_TSP_clean_FFTs' + file_series].shape
print(data_shape)
a.close()

(2050, 317109)


MC VS MD

In [11]:
mixed_number = 0
chosen = []
counter = 0
file_series = str(0)
while(mixed_number < 800):
    while(True):
        file1_indx = np.random.randint(0,(len(files_MC)-1) * 0.8) # 80%
        file2_indx = np.random.randint(0,(len(files_MD)-1) * 0.8) # 80%
        if str(file1_indx) + str('_') + str(file2_indx) not in chosen: # to prevent from choosing repeated files
            chosen.append(str(file1_indx) + str('_') + str(file2_indx))
            mixed_number += 1
            break
    wave1, samplerate = sf.read('TSP/48k/MC/' + files_MC[file1_indx])
    wave2, samplerate = sf.read('TSP/48k/MD/' + files_MD[file2_indx])
    if len(wave1) < len(wave2): # cropping data
        mixed = wave1 + wave2[0:len(wave1)]
        wave2 = wave2[0:len(wave1)]
    else:
        mixed = wave2 + wave1[0:len(wave2)]
        wave1 = wave1[0:len(wave2)]
    # framming and FFT and phase
    mixed_FFT, mixed_phase = STFT(mixed)
    wave1_FFT, wave1_phase = STFT(wave1)
    wave2_FFT, wave2_phase = STFT(wave2)  
    # concate wave 1 and wave 2 to have a 2 times dimention of each
    clean_FFTs = np.concatenate((wave1_FFT, wave2_FFT), axis = 0)
    clean_phases = np.concatenate((wave1_phase, wave2_phase), axis = 0)
    
    # writing files of TSP_mixed_FFT **** TSP_mixed_phase **** TSP_clean_FFTs **** TSP_clean_phases

    # mixed_FFT
    f_mixed_FFT = h5py.File('TSP/Organized/concatenated/Train_TSP_mixed_FFT' + file_series + '.hdf5', 'r+')     # open the file
    dset = f_mixed_FFT['Train_TSP_mixed_FFT' + file_series]                # load the data
    dset_shape = dset.shape
    dset.resize(dset_shape[1] + mixed_FFT.shape[1], axis=1)  
    dset[:,dset_shape[1]:] = np.abs(mixed_FFT)
    f_mixed_FFT.close() 
    
    # mixed_phase
    f_mixed_phase = h5py.File('TSP/Organized/concatenated/Train_TSP_mixed_phase' + file_series + '.hdf5', 'r+')     # open the file
    dset = f_mixed_phase['Train_TSP_mixed_phase' + file_series]                # load the data
    dset_shape = dset.shape
    dset.resize(dset_shape[1] + mixed_phase.shape[1], axis=1)  
    dset[:,dset_shape[1]:] = np.abs(mixed_phase)
    f_mixed_phase.close() 
    
    # clean_FFTs
    f_clean_FFTs = h5py.File('TSP/Organized/concatenated/Train_TSP_clean_FFTs' + file_series + '.hdf5', 'r+')     # open the file
    dset = f_clean_FFTs['Train_TSP_clean_FFTs' + file_series]                # load the data
    dset_shape = dset.shape
    dset.resize(dset_shape[1] + clean_FFTs.shape[1], axis=1)  
    dset[:,dset_shape[1]:] = np.abs(clean_FFTs)
    f_clean_FFTs.close() 
    
    # clean_phases
    f_clean_phases = h5py.File('TSP/Organized/concatenated/Train_TSP_clean_phases' + file_series + '.hdf5', 'r+')     # open the file
    dset = f_clean_phases['Train_TSP_clean_phases' + file_series]                # load the data
    dset_shape = dset.shape
    dset.resize(dset_shape[1] + clean_phases.shape[1], axis=1)  
    dset[:,dset_shape[1]:] = np.abs(clean_phases)
    f_clean_phases.close() 
    

    counter += 1


In [12]:
with open('TSP/Organized/concatenated/Train_selected_MC_MD.txt', 'w') as f:
    for item in chosen:
        f.write("%s\n" % item)

In [13]:
a = h5py.File('TSP/Organized/concatenated/Train_TSP_clean_FFTs'  + file_series + '.hdf5','r')
data_shape = a['Train_TSP_clean_FFTs' + file_series].shape
print(data_shape)
a.close()

(2050, 475371)
